In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
import xgboost as xgb
from sklearn.svm import SVC
from skbio.stats.ordination import pcoa
from skbio.stats.distance import permanova, DistanceMatrix
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.impute import SimpleImputer

def preprocess_and_pca(file_path, n_components=15):
    # Read the Excel file
    data = pd.read_excel(file_path, header=None)
    
    # Ignore the first row, the first column, and the last 3 columns
    data = data.iloc[1:, 1:-3]
    
    # Convert any non-numeric values to NaN
    data = data.apply(pd.to_numeric, errors='coerce')
    
    # Impute NaN values with the mean of each column
    imputer = SimpleImputer(strategy='mean')
    data_imputed = imputer.fit_transform(data)
    
    # Standardize the data
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data_imputed)
    
    # Perform PCA
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(scaled_data)
    
    # Create a DataFrame for the principal components
    columns = [f"PC{i+1}" for i in range(n_components)]
    pca_df = pd.DataFrame(data=principal_components, columns=columns)
    
    return pca_df, pca, scaler

# Paths to your datasets
hv_w_file_path = "Desktop/HV_Dataset.xlsx"
cc_w_file_path = "Desktop/CC_Dataset.xlsx"

# Number of PCA components
n_components = 15

# Preprocess and perform PCA
HV_w_df, hv_w_pca, hv_w_scaler = preprocess_and_pca(hv_w_file_path, n_components)
CC_w_df, cc_w_pca, cc_w_scaler = preprocess_and_pca(cc_w_file_path, n_components)

# Add labels to the DataFrames
HV_w_df['label'] = 0  # 0 for healthy patients
CC_w_df['label'] = 1  # 1 for cancer patients

# Concatenate the DataFrames
merged_df = pd.concat([HV_w_df, CC_w_df], axis=0)

# Split the data into features (X) and labels (y)
X = merged_df.drop(columns='label')
y = merged_df['label']

# Split into 80% training and 20% test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Initialize XGBoost and SVM models
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_estimators=100, max_depth=5)
svm_model = SVC(probability=True, random_state=42, C=1.0, kernel='rbf')  # SVM with RBF kernel

# Perform cross-validation with XGBoost and SVM on the training set
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

xgb_cv_predictions_proba = cross_val_predict(xgb_model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]
svm_cv_predictions_proba = cross_val_predict(svm_model, X_train, y_train, cv=cv, method='predict_proba')[:, 1]

# Ensemble predictions using simple averaging for training set
y_train_pred_ensemble_proba = (xgb_cv_predictions_proba + svm_cv_predictions_proba) / 2
y_train_pred_ensemble = (y_train_pred_ensemble_proba > 0.5).astype(int)

# Evaluate the ensemble model on the training set
accuracy_ensemble_train = accuracy_score(y_train, y_train_pred_ensemble)
precision_ensemble_train = precision_score(y_train, y_train_pred_ensemble)
recall_ensemble_train = recall_score(y_train, y_train_pred_ensemble)
f1_ensemble_train = f1_score(y_train, y_train_pred_ensemble)
auc_ensemble_train = roc_auc_score(y_train, y_train_pred_ensemble_proba)

print("\nEnsemble Model Performance on Training Set:")
print("Accuracy:", accuracy_ensemble_train)
print("Precision:", precision_ensemble_train)
print("Recall:", recall_ensemble_train)
print("F1 Score:", f1_ensemble_train)
print("AUC:", auc_ensemble_train)

# Fit the models on the entire training set
xgb_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

# Predict on the test set using the fitted models
xgb_test_predictions_proba = xgb_model.predict_proba(X_test)[:, 1]
svm_test_predictions_proba = svm_model.predict_proba(X_test)[:, 1]

# Ensemble predictions using simple averaging for test set
y_test_pred_ensemble_proba = (xgb_test_predictions_proba + svm_test_predictions_proba) / 2
y_test_pred_ensemble = (y_test_pred_ensemble_proba > 0.5).astype(int)

# Evaluate the ensemble model on the test set
accuracy_ensemble_test = accuracy_score(y_test, y_test_pred_ensemble)
precision_ensemble_test = precision_score(y_test, y_test_pred_ensemble)
recall_ensemble_test = recall_score(y_test, y_test_pred_ensemble)
f1_ensemble_test = f1_score(y_test, y_test_pred_ensemble)
auc_ensemble_test = roc_auc_score(y_test, y_test_pred_ensemble_proba)

print("\nEnsemble Model Performance on Test Set:")
print("Accuracy:", accuracy_ensemble_test)
print("Precision:", precision_ensemble_test)
print("Recall:", recall_ensemble_test)
print("F1 Score:", f1_ensemble_test)
print("AUC:", auc_ensemble_test)

# Combine datasets for beta diversity calculation
combined_data = pd.concat([HV_w_df.drop(columns='label'), CC_w_df.drop(columns='label')], axis=0)

# Calculate Bray-Curtis dissimilarity matrix
bray_curtis_matrix = pairwise_distances(combined_data, metric='braycurtis')

print("\nBray-Curtis Dissimilarity Matrix:")
print(pd.DataFrame(bray_curtis_matrix))

# Perform PERMANOVA
grouping_labels = np.concatenate([np.zeros(len(HV_w_df)), np.ones(len(CC_w_df))])  # Create grouping labels
grouping_labels_str = grouping_labels.astype(str)  # Convert labels to string for PERMANOVA
distance_matrix = DistanceMatrix(bray_curtis_matrix)  # Create DistanceMatrix object

permanova_results = permanova(distance_matrix, grouping_labels_str)
print("\nPERMANOVA Results:")
print(permanova_results)

# Calculate Beta Diversity (Bray-Curtis dissimilarity)
beta_diversity_healthy = bray_curtis_matrix[:len(HV_w_df), :len(HV_w_df)].mean()
beta_diversity_cancer = bray_curtis_matrix[len(HV_w_df):, len(HV_w_df):].mean()
beta_diversity_between_groups = bray_curtis_matrix[:len(HV_w_df), len(HV_w_df):].mean()

print("\nBeta Diversity for Healthy Patients:", beta_diversity_healthy)
print("Beta Diversity for Cancer Patients:", beta_diversity_cancer)
print("Beta Diversity Between Healthy and Cancer Patients:", beta_diversity_between_groups)

# Perform PCoA
pcoa_results = pcoa(bray_curtis_matrix)

# Plot PCoA results
def plot_pcoa(pcoa_results, labels):
    # Extract the first two PCoA coordinates
    pc1 = pcoa_results.samples['PC1']
    pc2 = pcoa_results.samples['PC2']

    # Create a scatter plot
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(pc1, pc2, c=labels, cmap='viridis', edgecolor='k', s=100, alpha=0.7)
    plt.xlabel(f'PC1 ({pcoa_results.proportion_explained[0]*100:.2f}% of variance)')
    plt.ylabel(f'PC2 ({pcoa_results.proportion_explained[1]*100:.2f}% of variance)')
    plt.title('PCoA of Microbiome Data (Bray-Curtis Dissimilarity)')
    plt.colorbar(scatter, ticks=[0, 1], label='Sample Label')
    plt.clim(-0.5, 1.5)
    plt.grid(False)  # Remove grid lines
    plt.show()

# Plot PCoA results with labels
labels = np.concatenate([np.zeros(len(HV_w_df)), np.ones(len(CC_w_df))])
plot_pcoa(pcoa_results, labels)

# Plot ROC curves for training and test sets
def plot_roc_curve(y_true, y_scores, title='ROC Curve'):
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    auc = roc_auc_score(y_true, y_scores)
    
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'AUC = {auc:.2f}')
    plt.plot([0, 1], [0, 1], color='grey', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

# Plot ROC curve for training set
plot_roc_curve(y_train, y_train_pred_ensemble_proba, title='ROC Curve - Training Set')

# Plot ROC curve for test set
plot_roc_curve(y_test, y_test_pred_ensemble_proba, title='ROC Curve - Test Set')


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

# Paths to your datasets
hv_file_path = "Desktop/HV_Dataset.xlsx"
cc_file_path = "Desktop/CC_Dataset.xlsx"

# Read the Excel files
healthy_data = pd.read_excel(hv_file_path, index_col=False)
cancer_data = pd.read_excel(cc_file_path, index_col=False)

# Exclude the first column from the datasets
healthy_data = healthy_data.iloc[:, 1:]
cancer_data = cancer_data.iloc[:, 1:]

# Function to compute Shannon diversity index and evenness for each patient
def compute_diversity_and_evenness(data):
    # Convert counts to proportions
    proportions = data.div(data.sum(axis=0), axis=1)
    # Compute Shannon diversity index
    diversity = -np.sum(proportions * np.log(proportions), axis=0)
    # Compute evenness
    evenness = diversity / np.log(data.shape[0])
    return diversity, evenness

# Compute diversity and evenness for healthy patients
healthy_diversity, healthy_evenness = compute_diversity_and_evenness(healthy_data)

# Compute diversity and evenness for cancer patients
cancer_diversity, cancer_evenness = compute_diversity_and_evenness(cancer_data)

# Statistical significance testing
diversity_stat, diversity_p = mannwhitneyu(healthy_diversity, cancer_diversity)
evenness_stat, evenness_p = mannwhitneyu(healthy_evenness, cancer_evenness)

# Create DataFrame for plotting
data = pd.DataFrame({
    'Group': ['Healthy'] * len(healthy_diversity) + ['Cancer'] * len(cancer_diversity),
    'Shannon Diversity': np.concatenate([healthy_diversity, cancer_diversity]),
    'Evenness': np.concatenate([healthy_evenness, cancer_evenness])
})

# Plotting
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 8), gridspec_kw={'hspace': 0.3})

# Box plot for Shannon Diversity Index
axs[0].boxplot([healthy_diversity, cancer_diversity], labels=['Healthy', 'Cancer'])
axs[0].set_ylabel('Shannon Diversity Index')
axs[0].set_title('Shannon Diversity Index for Each Patient\n(p = {:.3e})'.format(diversity_p))

# Box plot for Evenness
axs[1].boxplot([healthy_evenness, cancer_evenness], labels=['Healthy', 'Cancer'])
axs[1].set_ylabel('Evenness')
axs[1].set_title('Evenness for Each Patient\n(p = {:.3e})'.format(evenness_p))

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Paths to your datasets
hv_file_path = "Desktop/HV_Dataset.xlsx"
cc_file_path = "Desktop/CC_Dataset.xlsx"

# Read the Excel files
healthy_data = pd.read_excel(hv_file_path, index_col=False)
cancer_data = pd.read_excel(cc_file_path, index_col=False)

# Define a function to preprocess the data at the genus level
def preprocess_genus_level(data):
    # Group the data at the genus level (assuming first column is 'Microbiome at Genus Level')
    genus_data = data.groupby(data.columns[0]).sum()
    return genus_data

# Preprocess the data at the genus level for both datasets
healthy_genus_data = preprocess_genus_level(healthy_data)
cancer_genus_data = preprocess_genus_level(cancer_data)

# Compute the mean abundance and standard error for each genus across all patients
mean_abundance_healthy = healthy_genus_data.mean(axis=1)
mean_abundance_cancer = cancer_genus_data.mean(axis=1)

# Compute the standard error
std_error_healthy = healthy_genus_data.std(axis=1) / (len(healthy_genus_data.columns) ** 0.5)
std_error_cancer = cancer_genus_data.std(axis=1) / (len(cancer_genus_data.columns) ** 0.5)

# Find the top 5 genera with the highest differences in abundance between healthy and cancer datasets
top_diff_genera = (mean_abundance_cancer - mean_abundance_healthy).nlargest(5)

# Create a DataFrame to hold the abundance values for these top 5 genera
top_genera_abundance = pd.DataFrame({
    'Healthy': mean_abundance_healthy[top_diff_genera.index],
    'Cancer': mean_abundance_cancer[top_diff_genera.index],
    'Healthy_SE': std_error_healthy[top_diff_genera.index],
    'Cancer_SE': std_error_cancer[top_diff_genera.index]
})

# Plotting
fig, ax = plt.subplots(figsize=(12, 8))
bar_width = 0.35
indices = range(len(top_genera_abundance))

# Create bars for Healthy and Cancer with error bars
bars1 = ax.bar([i - bar_width/2 for i in indices], top_genera_abundance['Healthy'], bar_width,
               label='Healthy', color='skyblue', yerr=top_genera_abundance['Healthy_SE'], capsize=5)
bars2 = ax.bar([i + bar_width/2 for i in indices], top_genera_abundance['Cancer'], bar_width,
               label='Cancer', color='salmon', yerr=top_genera_abundance['Cancer_SE'], capsize=5)

# Adding labels
ax.set_xlabel('Genus')
ax.set_ylabel('Mean Abundance')
ax.set_title('Comparison of Top 5 Genera with Highest Differences in Abundance')
ax.set_xticks(indices)
ax.set_xticklabels(top_genera_abundance.index, rotation=45, ha='right')
ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Paths to your datasets
hv_file_path = "Desktop/HV_Dataset.xlsx"
cc_file_path = "Desktop/CC_Dataset.xlsx"

# Read the Excel files
healthy_data = pd.read_excel(hv_file_path, index_col=False)
cancer_data = pd.read_excel(cc_file_path, index_col=False)

# Function to compute average abundance across all patients
def compute_average_abundance(data):
    data = data.iloc[:, 1:]  # Ignore the first column (assumed to be labels)
    # Compute mean abundance for each microbiome across all patients
    mean_abundance = data.mean(axis=1)
    # Sort by abundance and select top 15
    mean_abundance_top15 = mean_abundance.sort_values(ascending=False).head(15)
    return mean_abundance_top15

# Compute average abundance for healthy and cancer patients
healthy_mean_abundance_top15 = compute_average_abundance(healthy_data)
cancer_mean_abundance_top15 = compute_average_abundance(cancer_data)

# Extract labels from the first column of the original data
healthy_labels = healthy_data.iloc[healthy_mean_abundance_top15.index, 0].values
cancer_labels = cancer_data.iloc[cancer_mean_abundance_top15.index, 0].values

# Combine the top labels from both datasets
top_labels = list(set(healthy_labels).union(set(cancer_labels)))

# Create a DataFrame for the combined top labels
combined_abundance = pd.DataFrame(index=top_labels)

# Fill in the abundance values, setting missing values to 0
combined_abundance['Healthy'] = healthy_data.set_index(healthy_data.columns[0]).reindex(top_labels).mean(axis=1).fillna(0)
combined_abundance['Cancer'] = cancer_data.set_index(cancer_data.columns[0]).reindex(top_labels).mean(axis=1).fillna(0)

# Normalize the abundances to sum up to 1 for each group
combined_abundance['Healthy'] = combined_abundance['Healthy'] / combined_abundance['Healthy'].sum()
combined_abundance['Cancer'] = combined_abundance['Cancer'] / combined_abundance['Cancer'].sum()

# Plotting
fig, ax = plt.subplots(figsize=(12, 8))
bar_width = 0.6
indices = range(len(combined_abundance.columns))

# Create stacked bars
bars = []
colors = plt.cm.tab20.colors  # Using a colormap for distinct colors
for i, bacteria in enumerate(combined_abundance.index):
    if i == 0:
        bars.append(ax.bar(indices, combined_abundance.loc[bacteria], bar_width, label=bacteria, color=colors[i % len(colors)]))
    else:
        bars.append(ax.bar(indices, combined_abundance.loc[bacteria], bar_width, bottom=combined_abundance.iloc[:i].sum(), label=bacteria, color=colors[i % len(colors)]))

# Adding labels
ax.set_xlabel('Group')
ax.set_ylabel('Relative Abundance')
ax.set_title('Top Abundant Bacteria in Healthy and Cancer Patients')
ax.set_xticks(indices)
ax.set_xticklabels(['Healthy', 'Cancer'])
ax.legend(title='Bacteria', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()
